# Big-Data

Description

In [ ]:
# Install necessary libraries to run this IPython Notebook
!pip install folium

## Treatment of data

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

In [ ]:
# Load spreadsheet
spreadsheet = pd.ExcelFile('datasets/imd_student_blind.xlsx')

# Load a sheet into a DataFrame by index
imd_student_blind_df = spreadsheet.parse(0)

In [ ]:
# Replace 'disciplina_ID' for its abreviation name

classes_map = {
    0: 'DISC_0',
    1: 'DISC_1',
    2: 'DISC_2',
    3: 'DISC_3',
    4: 'DISC_4',
    5: 'DISC_5',
    6: 'DISC_6'
}

imd_student_blind_df['disciplina_ID'].replace(classes_map, inplace = True)

In [ ]:
# Print the head of the DataFrame df
imd_student_blind_df.head()

In [ ]:
# Print the columns' name
imd_student_blind_df.columns

In [ ]:
imd_student_blind_df.shape

In [ ]:
# Load geolocation extracted data
students_geolocation = pd.read_csv('datasets/students_geolocation.csv')

In [ ]:
students_geolocation.info()

In [ ]:
students_geolocation.head()

In [ ]:
grouped_df = imd_student_blind_df.groupby('a_ID')

students_summary_df = grouped_df['nota'].agg({
    'grades_mean': 'mean',
    'grades_amount': 'count',
}).reset_index()

students_summary_df.rename(
    columns = {
        'a_ID' : 'id'
    },
    inplace = True
)

students_summary_df['distance'] = students_geolocation[students_geolocation['id'] == students_summary_df['id']]['distance']
students_summary_df['lat'] = students_geolocation[students_geolocation['id'] == students_summary_df['id']]['lat']
students_summary_df['lng'] = students_geolocation[students_geolocation['id'] == students_summary_df['id']]['lng']


In [ ]:
students_summary_df.head()

In [ ]:
grades_x_distance = pd.DataFrame({
    'distance': students_summary_df['distance'],
    'grades_mean': students_summary_df['grades_mean']
})

# Drop NaN values
grades_x_distance.dropna(inplace = True)

# Get only entries that have less than 30km from distance
grades_x_distance = grades_x_distance[grades_x_distance['distance'] < 30]

In [ ]:
grades_x_distance.info()

## Analysis

In [ ]:
# Import modules from Bokeh visualization library
from bokeh.charts import Scatter, BoxPlot, Bar, color
from bokeh.layouts import gridplot, row
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource

In [ ]:
imd_student_blind_df[['a_ID', 'ano_ingresso']].drop_duplicates().groupby('ano_ingresso').count()

In [ ]:
id_by_year = imd_student_blind_df[['a_ID', 'ano_ingresso']].drop_duplicates().groupby('ano_ingresso').count()
id_by_year

p = Bar(id_by_year, values='a_ID', xlabel='Year', ylabel='Total', 
        title="Students per year", color='a_ID')

output_notebook()
show(p)

In [ ]:
'''
data_groupby = ['ano_disciplina', 'periodo_disciplina', 'disciplina_ID']
nota = imd_student_blind_df.groupby(data_groupby)['nota']
nota.mean()
'''
# Gráfico
graph_lines = figure(plot_width=900, plot_height=300, title="Média de nota de cada disciplina, por período letivo")
colors = ['orange', 'blue', 'red', 'green', 'yellow', 'pink', 'purple', 'brown']
i = 0

# Array
ano_periodo_select = ['ano_disciplina', 'periodo_disciplina']

# Imprimir linhas de cada disciplina
for disciplina_ID in imd_student_blind_df['disciplina_ID'].unique():
    # Dados da disciplina
    data_disciplina = imd_student_blind_df[imd_student_blind_df['disciplina_ID'] == disciplina_ID]
    # Percorre o ano e o período
    ano_periodo = data_disciplina[ano_periodo_select].drop_duplicates()
    xline = []
    yline = []
    
    for ano in ano_periodo['ano_disciplina'].sort_values().unique():
        periodos_data = data_disciplina[data_disciplina['ano_disciplina'] == ano][['periodo_disciplina', 'nota']].sort_values('periodo_disciplina')
        for periodo in ano_periodo['periodo_disciplina'].sort_values().unique():
            # Se foi no primeiro ou segundo período
            if(periodo == 2):
                periodo_graph = 5
            else:
                periodo_graph = 0
            # X
            ap = "{0}.{1}".format(ano, periodo_graph)
            xline.append(float(ap))
            # Y
            media_periodo = periodos_data[periodos_data['periodo_disciplina'] == periodo]['nota'].mean()
            yline.append(float(media_periodo))
    # Data source para criar a linha
    line_disciplina_ds = ColumnDataSource(
        data=dict(
            x=xline,
            y=yline
        )
    )
    title = '{0}'.format(disciplina_ID)
    graph_lines.line('x', 'y', line_width=3, source=line_disciplina_ds, legend=title, color=colors[i])
    graph_lines.circle(xline, yline, color=colors[i], size=6, alpha=1.0)
    i += 1 # Avança no contador de cor
    
output_notebook()
show(graph_lines)

In [ ]:
p_grades_x_distance = Scatter(
        grades_x_distance,
        x='grades_mean',
        y='distance', 
        xlabel='Grades Mean',
        ylabel='Distance from IMD in kilometers',
        title='Grades Mean vs Distance from IMD',
        plot_width=800, plot_height=400
)

output_notebook()

show(p_grades_x_distance)

In [ ]:
# Create a coordinates list with the filtered data and show the heatmap

coordinates = []

for i in range(len(students_summary_df)):
    if all(~np.isnan([students_summary_df.ix[i,'lat'], students_summary_df.ix[i,'lng']])):
        coordinates.append([students_summary_df.ix[i,'lat'], students_summary_df.ix[i,'lng'], students_summary_df.ix[i,'grades_mean']])
        
f_map = folium.Map(
    location = [-5.791659, -35.28385],
    zoom_start = 11
)

HeatMap(coordinates).add_to(f_map)

f_map

## Key subjects per semester

In [ ]:
additional_column = pd.Series(
    None,
    index = imd_student_blind_df.index,
    name  = 'semester_code'
)
imd_student_blind_df = imd_student_blind_df.join(additional_column)

In [ ]:
for i in imd_student_blind_df.index:
    year = imd_student_blind_df.loc[i, 'ano_disciplina']
    semester = imd_student_blind_df.loc[i, 'periodo_disciplina']
    imd_student_blind_df.loc[i, 'semester_code'] = str(year) + '.' + str(semester)

In [ ]:
students_by_semester_df = imd_student_blind_df.groupby(['a_ID', 'semester_code'])['disciplina_ID'].unique().reset_index()

In [ ]:
def enrollments_per_semester(semester_index):
    semester_df = students_by_semester_df.groupby(['a_ID']).nth(semester_index).reset_index()

    # Rebuild the dataframe with one subject per line
    rows = []

    _ = semester_df.apply(
        lambda row: 
            [rows.append([row['a_ID'], row['semester_code'], d]) for d in row['disciplina_ID']],
         axis=1
    )

    semester_df = pd.DataFrame(
        rows,
        columns=['a_ID', 'semester_code', 'disciplina_ID']
    ).set_index(['a_ID', 'semester_code'])
    semester_df.reset_index(inplace = True)

    return semester_df.groupby('disciplina_ID')['a_ID'].count().reset_index()

In [ ]:
primary_semester_group_df = enrollments_per_semester(0)
secondary_semester_group_df = enrollments_per_semester(1)

In [ ]:
def semester_enrollments_bar(df, title):
    return Bar(
            df,
            label  = 'disciplina_ID',
            values = 'a_ID',
            title  = title,
            xlabel = 'Subjects',
            ylabel = 'Total enrollments',
            color  = 'disciplina_ID'
    )


primary_semester_bar = semester_enrollments_bar(
    primary_semester_group_df, 'Students per subject in the primary semester'
)

secondary_semester_bar = semester_enrollments_bar(
    secondary_semester_group_df, 'Students per subject in the secondary semester'
)

r = row([primary_semester_bar, secondary_semester_bar], sizing_mode='scale_width')

output_notebook()

show(r)

In [ ]:
# Based on main subjects in the primary semester set its dependents on secondary semester

SECONDARY_SUBJECTS = ['DISC_0', 'DISC_3', 'DISC_4', 'DISC_5', 'DISC_6']

## Dependents subjects analysis

In [ ]:
def get_group_by_grade(grade):
    if(grade >= 5 and grade < 6):
        return '5 - 6'
    if(grade >= 6 and grade < 7):
        return '6 - 7'
    if(grade >= 7 and grade < 8):
        return '7 - 8'
    if(grade >= 8 and grade < 9):
        return '8 - 9'
    if(grade >= 9):
        return '9 - 10'

In [ ]:
# Add the lib package to the system path, so that we can include SubjectConnections from there.

import os
import sys
module_path = os.path.abspath(os.path.join('./'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from lib.subject_connections import SubjectConnections

In [ ]:
# Generate bar chart by students grouped df

def students_grouped_bar(df, title):
    return Bar(
            df,
            label  = 'grade_group',
            values = 'amount',
            title  = title,
            xlabel = 'Grade groups',
            ylabel = 'Total approved students',
            color  = 'grade_group',
            legend='top_right',
            plot_height=300
            
    )

In [ ]:
# Helper method to generate boxplot

def generate_boxplot(df, base_subject_key, subject_key): 
    return BoxPlot(
        df,
        values = '{}_grade'.format(subject_key),
        label  = 'grade_group',
        color  = 'grade_group',
        xlabel = '{} grade groups'.format(base_subject_key),
        ylabel = '{} grades'.format(subject_key), 
        title  = '{} student grades (first enrollment) grouped by {} grades'.format(subject_key, base_subject_key)
    )

In [ ]:
# Generate boxplot list based on SubjectConnections instance

def dependent_subjects_list(list, subj_connection, base_subject):
    boxplot_list = []

    for subject in list:
        column_name = '{}_grade'.format(subject)

        students_df = imd_student_blind_df[imd_student_blind_df['disciplina_ID'] == subject]
        df = subj_connection.obtain_values_from(students_df, column_name, ['grade_group'])
        
        grade_groups_mean_df = df.groupby('grade_group')[column_name].mean().reset_index()
        
        box = generate_boxplot(df, base_subject, subject)
        box.line(x = grade_groups_mean_df['grade_group'], y = grade_groups_mean_df[column_name], line_width = 2, line_color = '#2C3E50')
        box.circle(x = grade_groups_mean_df['grade_group'], y = grade_groups_mean_df[column_name], color = "#F1C40F", size = 5)

        boxplot_list.append(box)
    
    return boxplot_list

### DISC_1

In [ ]:
DISC_1_student = imd_student_blind_df['disciplina_ID'] == 'DISC_1'
has_approved_status = imd_student_blind_df['status.disciplina'] == 'Aprovado'

DISC_1_students_df = imd_student_blind_df[DISC_2_student & has_approved_status]

DISC_1_connections = SubjectConnections(RPMTI_students_df, 'a_ID')
DISC_1_connections.parse_column('nota', 'grade_group', get_group_by_grade)

In [ ]:
total_approved_DISC_1_by_group = DISC_1_connections.base_df.groupby('grade_group', as_index=False)['a_ID'].agg({
    'amount': 'count'
})

b = students_grouped_bar(total_approved_DISC_1_by_group, 'Total approved students on DISC_1 by grade group')

output_notebook()
show(b)

In [ ]:
boxplots = dependent_subjects_list(
    SECONDARY_SUBJECTS, DISC_1_connections, 'DISC_1'
)

grid = gridplot(boxplots, ncols=2, plot_width=460, plot_height=400)

output_notebook()
show(grid)

### DISC_2

In [ ]:
DISC_2_student = imd_student_blind_df['disciplina_ID'] == 'DISC_2'
has_approved_status = imd_student_blind_df['status.disciplina'] == 'Aprovado'

DISC_2_students_df = imd_student_blind_df[DISC_2_student & has_approved_status]

DISC_2_connections = SubjectConnections(RPMTI_students_df, 'a_ID')
DISC_2_connections.parse_column('nota', 'grade_group', get_group_by_grade)

In [ ]:
total_approved_DISC_1_by_group = DISC_1_connections.base_df.groupby('grade_group', as_index=False)['a_ID'].agg({
    'amount': 'count'
})

b = students_grouped_bar(total_approved_DISC_1_by_group, 'Total approved students on DISC_1 by grade group')

output_notebook()
show(b)

In [ ]:
boxplots = dependent_subjects_list(
    SECONDARY_SUBJECTS, DISC_2_connections, 'DISC_2'
)

grid = gridplot(boxplots, ncols=2, plot_width=460, plot_height=400)

output_notebook()
show(grid)

## Students outside the course

In [ ]:
CANCELADO = imd_student_blind_df['status'] == 'CANCELADO'
TRANCADO = imd_student_blind_df['status'] == 'TRANCADO'

students_out_df = imd_student_blind_df[CANCELADO | TRANCADO]

### By total of subject statuses

In [ ]:
DISCIPLINA_APROVADO = students_out_df['status.disciplina'] == 'Aprovado'
DISCIPLINA_REPROVADO = students_out_df['status.disciplina'] == 'Reprovado'

approved_students_df = students_out_df[DISCIPLINA_APROVADO].groupby('disciplina_ID')['a_ID'].agg({
    'total': 'count'
}).reset_index()

additional_column = pd.Series(
    'Aprovado',
    index = approved_students_df.index,
    name  = 'status'
)
approved_students_df = approved_students_df.join(additional_column)

disapproved_students_df = students_out_df[DISCIPLINA_REPROVADO].groupby('disciplina_ID')['a_ID'].agg({
    'total': 'count'
}).reset_index()

additional_column = pd.Series(
    'Reprovado',
    index = disapproved_students_df.index,
    name  = 'status'
)
disapproved_students_df = disapproved_students_df.join(additional_column)

students_out_statuses_df = pd.concat([approved_students_df, disapproved_students_df], ignore_index = True)

In [ ]:
p = Bar(
        students_out_statuses_df,
        label  = 'disciplina_ID',
        values = 'total',
        group  = 'status',
        title  ='Students outside the course by total of subject statuses',
        legend = 'top_center',
        xlabel = 'Subjects',
        ylabel = 'Total statuses', 
        color  = color(columns = 'status', palette = ['#7BC560', '#F55666'])
)

output_notebook()

show(p)